In [1]:
import pandas as pd
 

import pandas as pd
import numpy as np
 
from datetime import datetime, timedelta

In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
 


In [89]:
df=pd.read_csv("data_imu_birlestirilmis.csv")

In [90]:
df

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,not_falling
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,not_falling
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,not_falling
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,not_falling
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,not_falling
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,not_falling
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,not_falling
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,not_falling
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,not_falling


In [91]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from sklearn.preprocessing import LabelEncoder,StandardScaler
from sklearn.model_selection import train_test_split

In [92]:
label_encoder=LabelEncoder()

In [93]:
df["Class Name"]=label_encoder.fit_transform(df["Class Name"])

In [94]:
print(label_encoder.classes_)



['falling' 'not_falling']


In [95]:
df

,Unnamed: 0,ax,ay,az,gx,gy,gz,timestamp,Class Name
0,0,1.091892,8.707838,1.765225,-0.036396,0.063694,-0.036396,1900-01-01 17:44:42.000000,1
1,1,1.083568,8.732087,1.765318,-0.036781,0.064346,-0.036686,1900-01-01 17:44:42.020000,1
2,2,1.075247,8.756325,1.765411,-0.037166,0.064996,-0.036976,1900-01-01 17:44:42.040000,1
3,3,1.066932,8.780539,1.765503,-0.037551,0.065642,-0.037265,1900-01-01 17:44:42.060000,1
4,4,1.058625,8.804720,1.765596,-0.037936,0.066281,-0.037553,1900-01-01 17:44:42.080000,1
...,...,...,...,...,...,...,...,...,...
51645,51645,1.611913,8.047413,-3.202833,-0.106623,0.084735,-0.050371,1900-01-01 17:59:15.899794,1
51646,51646,1.626502,8.043946,-3.201882,-0.118920,0.123886,-0.071806,1900-01-01 17:59:15.919794,1
51647,51647,1.641042,8.040466,-3.200937,-0.131261,0.163164,-0.093307,1900-01-01 17:59:15.939794,1
51648,51648,1.655544,8.036974,-3.199998,-0.143635,0.202538,-0.114858,1900-01-01 17:59:15.959793,1


In [96]:
X = df[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values   
y = df['Class Name'].values

In [97]:
scaler = StandardScaler()
X = scaler.fit_transform(X)

In [98]:
window_size = 25

In [99]:
def create_windowed_data(X, y, window_size):
    Xs, ys = [], []
    for i in range(len(X) - window_size + 1):
        Xs.append(X[i:i+window_size])   
        ys.append(y[i+window_size-1])   
    return np.array(Xs), np.array(ys)

In [100]:
X_windowed, y_windowed = create_windowed_data(X, y, window_size)

In [101]:
X_windowed.shape

(51626, 25, 6)

In [102]:
X_train, X_test, y_train, y_test = train_test_split(X_windowed, y_windowed, test_size=0.2, random_state=3)

In [103]:
X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32).unsqueeze(1)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32).unsqueeze(1)

In [104]:
y_train.shape

torch.Size([41300, 1])

In [105]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_train, y_train, X_test, y_test = X_train.to(device), y_train.to(device), X_test.to(device), y_test.to(device)

In [106]:
class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size, dropout_prob):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout_prob)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
    
    def forward(self, x):
        h_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        c_0 = torch.zeros(num_layers, x.size(0), hidden_size).to(device)   
        
        out, _ = self.lstm(x, (h_0, c_0))  # LSTM forward
        out = self.fc(out[:, -1, :])  # Son zaman adımının çıktısını alır
        out = self.sigmoid(out)  # Sigmoid ile iki sınıfa ayırır
        return out


In [107]:
input_size = X_train.shape[2]  # ax, ay, az, gx, gy, gz (6 özellik)
hidden_size = 24 # Gizli katman boyutu
num_layers = 2  # LSTM katman sayısı
output_size = 1  # Binary classification
dropout_prob = 0.5  # Dropout oranı
torch.cuda.empty_cache()

In [108]:
X_train.shape[2]

6

In [109]:
model = LSTMModel(input_size, hidden_size, num_layers, output_size, dropout_prob).to(device)
criterion = nn.BCELoss()   
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [110]:
 
num_epochs = 1400
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

 
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            y_pred_class = (y_pred > 0.5).float()
            accuracy = (y_pred_class == y_test).sum().item() / y_test.size(0)
            print(f'Test Accuracy: {accuracy:.4f}')

Epoch [10/1400], Loss: 0.6940
Test Accuracy: 0.6133
Epoch [20/1400], Loss: 0.6727
Test Accuracy: 0.6905
Epoch [30/1400], Loss: 0.6428
Test Accuracy: 0.6908
Epoch [40/1400], Loss: 0.5982
Test Accuracy: 0.6917
Epoch [50/1400], Loss: 0.5872
Test Accuracy: 0.6982
Epoch [60/1400], Loss: 0.5754
Test Accuracy: 0.7147
Epoch [70/1400], Loss: 0.5677
Test Accuracy: 0.7207
Epoch [80/1400], Loss: 0.5585
Test Accuracy: 0.7242
Epoch [90/1400], Loss: 0.5471
Test Accuracy: 0.7272
Epoch [100/1400], Loss: 0.5367
Test Accuracy: 0.7295
Epoch [110/1400], Loss: 0.5279
Test Accuracy: 0.7349
Epoch [120/1400], Loss: 0.5205
Test Accuracy: 0.7436
Epoch [130/1400], Loss: 0.5134
Test Accuracy: 0.7509
Epoch [140/1400], Loss: 0.5051
Test Accuracy: 0.7592
Epoch [150/1400], Loss: 0.4956
Test Accuracy: 0.7631
Epoch [160/1400], Loss: 0.4859
Test Accuracy: 0.7682
Epoch [170/1400], Loss: 0.4747
Test Accuracy: 0.7737
Epoch [180/1400], Loss: 0.4614
Test Accuracy: 0.7868
Epoch [190/1400], Loss: 0.4482
Test Accuracy: 0.7976
Ep

In [111]:
num_epochs = 1400
batch_size = 32

for epoch in range(num_epochs):
    model.train()
    optimizer.zero_grad()
    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()
    
    if (epoch+1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
 
        model.eval()
        with torch.no_grad():
            y_pred = model(X_test)
            y_pred_class = (y_pred > 0.5).float()
            accuracy = (y_pred_class == y_test).sum().item() / y_test.size(0)
            print(f'Test Accuracy: {accuracy:.4f}')

Epoch [10/1400], Loss: 0.0807
Test Accuracy: 0.9703
Epoch [20/1400], Loss: 0.0805
Test Accuracy: 0.9711
Epoch [30/1400], Loss: 0.0798
Test Accuracy: 0.9726
Epoch [40/1400], Loss: 0.0808
Test Accuracy: 0.9718
Epoch [50/1400], Loss: 0.0780
Test Accuracy: 0.9712
Epoch [60/1400], Loss: 0.0787
Test Accuracy: 0.9725
Epoch [70/1400], Loss: 0.0766
Test Accuracy: 0.9716
Epoch [80/1400], Loss: 0.0777
Test Accuracy: 0.9718
Epoch [90/1400], Loss: 0.0790
Test Accuracy: 0.9720
Epoch [100/1400], Loss: 0.0761
Test Accuracy: 0.9735
Epoch [110/1400], Loss: 0.0763
Test Accuracy: 0.9732
Epoch [120/1400], Loss: 0.0747
Test Accuracy: 0.9730
Epoch [130/1400], Loss: 0.0757
Test Accuracy: 0.9725
Epoch [140/1400], Loss: 0.0755
Test Accuracy: 0.9731
Epoch [150/1400], Loss: 0.0713
Test Accuracy: 0.9719
Epoch [160/1400], Loss: 0.0745
Test Accuracy: 0.9730
Epoch [170/1400], Loss: 0.0713
Test Accuracy: 0.9740
Epoch [180/1400], Loss: 0.0702
Test Accuracy: 0.9718
Epoch [190/1400], Loss: 0.0720
Test Accuracy: 0.9734
Ep

2400 epoch batcsize =32  window_size=25 (sampling rate yarısı), accurac 0.9853 

In [112]:
MODEL_PATH = "lstm_falling_detection_modelv5_9853.pth"
torch.save(model.state_dict(), MODEL_PATH)
print(f"Model saved to {MODEL_PATH}")

Model saved to lstm_falling_detection_modelv5_9853.pth


In [113]:

torch.save({
    'epoch': num_epochs,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': loss.item(),
}, 'lstm_model_checkpoint_model_v5_9853.pth')
print("Model ve optimizer durumları kaydedildi.")

Model ve optimizer durumları kaydedildi.


In [67]:
from torchsummary  import summary

In [74]:
model

LSTMModel(
  (lstm): LSTM(6, 24, num_layers=2, batch_first=True, dropout=0.5)
  (fc): Linear(in_features=24, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)